In [1]:
%load_ext line_profiler

In [2]:


import sys
import pyvista as pv
from pyvista import PolyData, Plotter
import numpy as np
from irregular_object_packing.mesh.transform import scale_and_center_mesh, scale_to_volume
from irregular_object_packing.mesh.utils import print_mesh_info
from irregular_object_packing.packing import initialize, nlc_optimisation as nlc 
from irregular_object_packing.cat import chordal_axis_transform as cat
from irregular_object_packing.packing.optimizer import SimConfig, Optimizer, default_optimizer_config
from irregular_object_packing.packing.plots import generate_tinted_colors
from importlib import reload

In [3]:
DATA_FOLDER = "../data/mesh/"

optimizer = default_optimizer_config(N=5, mesh_dir=DATA_FOLDER,)
optimizer.setup()


In [4]:
optimizer.run()


scale	:   0%|          | 0/9 [00:00<?, ?it/s]

Iteration	:   0%|          | 0/100 [00:00<?, ?it/s]

Object	:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/maurits/code/cls/thesis/irregular-object-packing/irregular_object_packing/cat/utils.py:120: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'points' of function 'compute_face_unit_normal'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../irregular_object_packing/cat/utils.py", line 70:
@jit(nopython=True, cache=True)
def compute_face_unit_normal(points, v_i):
^

  vertex_face_normal[2] = compute_face_unit_normal(face_vertices, obj_point)
/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'face_vertices' of function 'create_face_normal'.

For more information visit https://numba.readthedocs.io/en/s

         12473468 function calls (12095830 primitive calls) in 8.688 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
452672/430744    1.036    0.000    1.346    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    13524    0.705    0.000    0.897    0.000 ffi.py:150(__call__)
       90    0.397    0.004    0.397    0.004 {method 'Update' of 'vtkmodules.vtkCommonExecutionModel.vtkAlgorithm' objects}
   364484    0.384    0.000    2.277    0.000 pyvista_ndarray.py:40(__array_finalize__)
    19482    0.372    0.000    6.918    0.000 chordal_axis_transform.py:59(split_and_process)
   364484    0.353    0.000    0.651    0.000 dataset_adapter.py:275(__array_finalize__)
    21898    0.312    0.000    0.480    0.000 utils.py:8(sort_by_occurrance)
    14704    0.271    0.000    1.668    0.000 tetrahedral_split.py:83(split_2_3331)
1769325/1767822    0.224    0.000    0.280    0.000 {built-in method built

## Visualisation

The following cell shows a visualisation of the final result.


In [5]:
import seaborn as sns
plotter = Plotter()
plotter.enable_depth_peeling()
tints = generate_tinted_colors(optimizer.n_objs)
pv.set_plot_theme("document")
sns.set_style('whitegrid')
tints = sns.color_palette('deep')

def cclip(mesh: pv.PolyData):
    return mesh.clip_closed_surface('-z', origin=(0,0,0.5))

def oclip(mesh: pv.PolyData):
    return mesh.clip('z', origin=(0,0,0.5), invert=True)

plotter.add_mesh(oclip(optimizer.container), color="white", opacity=0.2)
for i, mesh in enumerate(optimizer.final_cat_meshes()):
    try:
        plotter.add_mesh(oclip(mesh), color='yellow', opacity=0.5)
    except:
        pass

for i, mesh in enumerate(optimizer.current_meshes()):
    try:
        plotter.add_mesh(cclip(mesh), color=tints[i], opacity=1)
    except:
        pass


# plotter.add_mesh(optimizer.container, color="grey", opacity=0.2)
plotter.camera.tight()
plotter.view_xy()

plotter.save_graphic("packing_slice2.pdf")


In [6]:
# Plotting the final step before and after optimization
from irregular_object_packing.packing.plots import plot_simulation_scene
def plot(plotter, meshes, cat_meshes=None, container=None):
    for i in range(len(meshes)):
        plotter.add_mesh(meshes[i], opacity=1, color=tints[i],)
        if cat_meshes is not None:
            plotter.add_mesh(cat_meshes[i], opacity=0.4, color="yellow", )
    plotter.add_mesh(container, color="white", opacity=0.2,)
    plotter.camera.tight()
    plotter.view_xy()

In [7]:
meshes = optimizer.current_meshes()
cat_meshes = optimizer.final_cat_meshes()
container = optimizer.container
plotter = Plotter()
plot(plotter, meshes, cat_meshes, container)
plotter.save_graphic("packing_5_after_with_cat.pdf")
plotter.show()

plotter = Plotter()
plot(plotter, meshes, container=container)
plotter.save_graphic("packing_5_after_no_cat.pdf")

Widget(value="<iframe src='http://localhost:62029/index.html?ui=P_0x166c44730_1&reconnect=auto' style='width: …

In [8]:
meshes = optimizer.meshes_before(0)
cat_meshes = optimizer.cat_meshes(0)
container = optimizer.container
plotter = Plotter()
plot(plotter, meshes, container=container)
plotter.save_graphic("packing_5_start.pdf")
plotter.show()

Widget(value="<iframe src='http://localhost:62029/index.html?ui=P_0x166c44310_3&reconnect=auto' style='width: …